# Gravitational Lensing and Einstein Rings

Gravitational lensing bends light around massive objects, magnifying and distorting background sources. When the lens, source, and observer are nearly aligned, the light can form a symmetric arc or full Einstein ring.

## Project and PyAutoLens

PyAutoLens is an open-source lens modeling library. This notebook walks through a compact demo: a quick theory recap, a Fourier transform refresher, loading a public image, simulating an Einstein ring with PyAutoLens, adding noise, and cleaning it with a Wiener filter.

## Background and key formulas

- Lens equation (sky coords in radians): beta = theta - (D_ls / D_s) * alpha(theta), where D terms are angular diameter distances and alpha is the deflection angle.
- Einstein radius (for a circular isothermal lens): theta_E = 4 * pi * (sigma_v / c)^2 * (D_ls / D_s), the scale that sets the ring size.
- Surface mass density via convergence: kappa(theta) = Sigma(theta) / Sigma_crit with Sigma_crit = (c^2 / 4 / pi / G) * (D_s / (D_l * D_ls)).
- Fourier transform reminder: FFT maps an image into spatial-frequency space; filtering often happens there. Noise here is modeled as additive Gaussian white noise with variance sigma_n^2.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Quick Fourier transform demo
x = np.linspace(0, 2 * np.pi, 256)
signal = np.sin(3 * x) + 0.3 * np.sin(9 * x)
fft = np.fft.fft(signal)
freqs = np.fft.fftfreq(signal.size, d=x[1] - x[0])

fig, axes = plt.subplots(1, 2, figsize=(10, 3))
axes[0].plot(x, signal)
axes[0].set_title("Synthetic time-domain signal")
axes[0].set_xlabel("x")
axes[0].set_ylabel("amplitude")

axes[1].stem(freqs[: len(freqs) // 2], np.abs(fft)[: len(freqs) // 2], basefmt=" ")
axes[1].set_title("Magnitude of FFT")
axes[1].set_xlabel("frequency (1/rad)")

fig.tight_layout()
plt.show()

In [ ]:
# Uncomment the next line if running in a fresh environment.
# %pip install -q pyautolens[complete] matplotlib pillow scipy

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from scipy.signal import wiener

In [ ]:
tif_path = Path("potm2503a.tif")
if not tif_path.exists():
    raise FileNotFoundError(f"Expected image at {tif_path!s}")
img = Image.open(tif_path).convert("RGB")
img = img.resize((256, 256))
image_array = np.asarray(img) / 255.0

plt.figure(figsize=(5, 5))
plt.imshow(img)
plt.axis("off")
plt.title("potm2503a.tif (local example)")
plt.show()

This local TIF (`potm2503a.tif`) serves as the main example for strong lensing context. Below, we will generate a simplified Einstein-ring-like image with PyAutoLens and then explore how noise and filtering shape the data.

## Real TIFF reconstruction scaffold

Fill the placeholders below with real metadata before running. Required:
- `pixel_scales` (arcsec/pixel)
- `psf_fwhm_arcsec` (arcsec full width at half max) or supply a measured PSF kernel
- `background_rms` noise per pixel (same units as the data)
- `mask_radius_arcsec` to keep only the region around the ring

Optional: set `target_shape` to downsample for quick tests. This creates an `al.Imaging` dataset and mask ready for a PyAutoLens fit once real numbers are supplied.


In [ ]:
try:
    import autolens as al
except ImportError as exc:
    raise RuntimeError("Install PyAutoLens with `pip install pyautolens[complete]`.") from exc

PIXEL_SCALES = 0.05  # TODO: arcsec per pixel from the observation
PSF_FWHM_ARCSEC = 0.7  # TODO: arcsec FWHM of the PSF
BACKGROUND_RMS = 0.02  # TODO: per-pixel noise level in data units
MASK_RADIUS_ARCSEC = 3.0  # TODO: arcsec radius covering the Einstein ring

if any(v is None for v in [PIXEL_SCALES, PSF_FWHM_ARCSEC, BACKGROUND_RMS, MASK_RADIUS_ARCSEC]):
    raise ValueError("Fill in PIXEL_SCALES, PSF_FWHM_ARCSEC, BACKGROUND_RMS, MASK_RADIUS_ARCSEC.")

tif_path = Path("potm2503a.tif")
tif_image = Image.open(tif_path).convert("F")
data_native = np.asarray(tif_image, dtype="float32")

# Optional downsample to speed up early iterations (set to None to keep native resolution).
TARGET_SHAPE = None  # e.g., (512, 512)
if TARGET_SHAPE:
    tif_image = tif_image.resize(TARGET_SHAPE)
    data_native = np.asarray(tif_image, dtype="float32")

sigma_pixels = (PSF_FWHM_ARCSEC / PIXEL_SCALES) / 2.355
psf_shape = (21, 21)  # TODO: ensure this covers the PSF footprint
psf = al.Kernel2D.from_gaussian(
    shape_native=psf_shape, sigma=sigma_pixels, pixel_scales=PIXEL_SCALES
)

noise_native = np.full_like(data_native, fill_value=BACKGROUND_RMS, dtype="float32")

mask = al.Mask2D.unmasked(shape_native=data_native.shape, pixel_scales=PIXEL_SCALES)
data = al.Array2D.manual_native(array=data_native, mask=mask)
noise_map = al.Array2D.manual_native(array=noise_native, mask=mask)

imaging = al.Imaging(image=data, noise_map=noise_map, psf=psf)

mask = al.Mask2D.circular(
    shape_native=imaging.shape_native,
    pixel_scales=PIXEL_SCALES,
    radius=MASK_RADIUS_ARCSEC,
)
imaging = imaging.apply_mask(mask)

print(f"Imaging ready: shape={imaging.shape_native}, pixel_scales={PIXEL_SCALES}")


In [ ]:
# Minimal model skeleton; update priors once real metadata is known.
lens_galaxy_model = al.Model(
    al.Galaxy(
        redshift=0.5,
        mass=al.mp.Isothermal(
            centre=(0.0, 0.0),
            ell_comps=(0.0, 0.0),
            einstein_radius=1.4,  # TODO: set a prior mean/width
        ),
        light=al.lp.Sersic(
            centre=(0.0, 0.0),
            intensity=0.1,
            effective_radius=0.4,
            sersic_index=2.5,
        ),
    )
)

source_galaxy_model = al.Model(
    al.Galaxy(
        redshift=1.0,
        light=al.lp.Sersic(
            centre=(0.0, 0.0),
            intensity=0.5,
            effective_radius=0.2,
            sersic_index=1.5,
            ell_comps=(0.0, 0.0),
        ),
    )
)

model = al.Model(
    al.Tracer(galaxies=[lens_galaxy_model, source_galaxy_model])
)

analysis = al.AnalysisImaging(dataset=imaging)

search = al.Nautilus(
    path_prefix="output",
    name="tif_dummy_fit",
    n_live=100,  # increase for production runs
    number_of_cores=1,
)

# Uncomment to run once real metadata is filled in.
# result = search.fit(model=model, analysis=analysis)
# print(result.info)


In [ ]:
try:
    import autolens as al
except ImportError as exc:
    raise RuntimeError(
        "PyAutoLens is required here. Install with `pip install pyautolens[complete]`."
    ) from exc

grid = al.Grid2D.uniform(shape_native=(200, 200), pixel_scales=0.05)

lens_galaxy = al.Galaxy(
    redshift=0.5,
    mass=al.mp.Isothermal(
        centre=(0.0, 0.0),
        ell_comps=(0.05, 0.05),
        einstein_radius=1.4,
    ),
)

source_galaxy = al.Galaxy(
    redshift=1.0,
    light=al.lp.Sersic(
        centre=(0.08, 0.05),
        intensity=1.0,
        effective_radius=0.3,
        sersic_index=2.5,
        ell_comps=(0.1, -0.05),
    ),
)

tracer = al.Tracer(galaxies=[lens_galaxy, source_galaxy])
lensed_image = tracer.image_2d_from(grid=grid)
lensed_array = lensed_image.native

plt.figure(figsize=(5, 5))
plt.imshow(lensed_array, cmap="magma")
plt.axis("off")
plt.title("Simulated Einstein ring (PyAutoLens)")
plt.show()

**Key variables**
- `pixel_scales`: arcsec per pixel; smaller values capture finer structure at a higher computational cost.
- `ell_comps`: ellipticity components (e1, e2) that control axis ratio and orientation of the mass and light profiles.
- `einstein_radius`: sets the ring size; larger values push bright arcs farther from the center.
- `sersic_index`, `effective_radius`, `intensity`: describe the source brightness profile; higher index concentrates light toward the center.

In [ ]:
if "lensed_array" not in globals():
    raise RuntimeError("Run the PyAutoLens cell first to define `lensed_array`.")

lensed_norm = lensed_array / np.max(lensed_array)
rng = np.random.default_rng(0)
noisy = np.clip(lensed_norm + rng.normal(scale=0.08, size=lensed_norm.shape), 0, 1)
denoised = np.clip(wiener(noisy, mysize=5), 0, 1)

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(lensed_norm, cmap="magma")
axes[0].set_title("Clean simulation")
axes[0].axis("off")

axes[1].imshow(noisy, cmap="magma")
axes[1].set_title("With Gaussian noise")
axes[1].axis("off")

axes[2].imshow(denoised, cmap="magma")
axes[2].set_title("Wiener filtered")
axes[2].axis("off")

fig.tight_layout()
plt.show()

We added Gaussian white noise and then applied a Wiener filter that adapts the smoothing window to local variance. The noisy image shows speckling and washed-out arcs; the filtered result recovers a smoother ring while suppressing noise at the cost of slight blurring.

## Conclusion

You saw a minimal PyAutoLens setup that builds a lens-source model, produces an Einstein-ring-like image, and demonstrates how noise degrades the signal and how simple filtering helps recover it. The same workflow extends to real data: swap in survey images, fit mass profiles, and iterate on filtering strategies to maximize signal-to-noise.